In [ ]:
import biosppy.signals.ecg as ecg

In [ ]:
signals = []
with open("data/X_train.csv") as f_train:
    for line in f_train.readlines()[1:]:
        signals.append(list(map(int, line.split(',')[1:])))
y = []
with open("data/y_train.csv") as f:
    for line in f.readlines()[1:]:
        y.append(int(line.split(',')[1]))

In [ ]:
len(signals)

In [ ]:
ecg.ecg(signals[5], 300.0)

In [ ]:
r = ecg.ecg(signals[0], 300.0)
len(r[3])

In [ ]:
len(r[4][0])

In [ ]:
len(r[4])

In [ ]:
type0 = [signals[i] for i in range(len(y)) if y[i] == 0]
type1 = [signals[i] for i in range(len(y)) if y[i] == 1]
type2 = [signals[i] for i in range(len(y)) if y[i] == 2]
type3 = [signals[i] for i in range(len(y)) if y[i] == 3]

In [ ]:
for s in type1[:20]:
    print(len(ecg.ecg(s, 300.0)[4][0]))
    # rpeaks = ecg.ssf_segmenter(s, 300.0)[0]
    # print(len(ecg.extract_heartbeats(s, rpeaks, 300.0)[0]))

In [ ]:
def template(s):
    # print(s)
    # rpeaks = ecg.christov_segmenter(signal=s, sampling_rate=300.0)[0]
    # print(rpeaks)
    # return ecg.extract_heartbeats(s, rpeaks=rpeaks, sampling_rate=300.0)[0]
    return ecg.ecg(s, sampling_rate=300.0, show=False)[4]

In [ ]:
X = []
cnt = 0
for s in signals:
    print(cnt)
    cnt += 1
    beats = template(s)
    temp = [0 for i in range(len(beats[0]))]
    for beat in beats:
        for i, val in enumerate(beat):
            temp[i] += val
    for i, val in enumerate(temp):
        temp[i] = val / len(beats)
    X.append(temp)
X

In [ ]:
y

In [ ]:
with open("template.csv", "w") as f:
    s = "Id"
    for i in range(150):
        s += ",x%d"%i
    s += "\n"
    f.write(s)
    for x in X:
        s = ','.join(map(str,x)) + "\n"
        f.write(s)


In [ ]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [ ]:
# train_X = np.array(X[:int(len(X)*0.7)])
# train_Y = np.array(y[:int(len(X)*0.7)])
# eval_X = np.array(X[int(len(X)*0.7):])
# eval_Y = np.array(y[int(len(X)*0.7):])
train_X, eval_X, train_Y, eval_Y = train_test_split(X, y, test_size=0.33, random_state=42)
xg_train = xgb.DMatrix(train_X, label=train_Y)
xg_eval = xgb.DMatrix(eval_X, label=eval_Y)

# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.2
param['gamma'] = 1.0
param['max_depth'] = 6
param['silent'] = 1
param['subsample'] = 0.8
param['colsample_bytree'] = 0.9
param['min_child_weight'] = 20
param['num_class'] = 4


watchlist = [(xg_train, 'train'), (xg_eval, 'eval')]
num_round = 60
bst = xgb.train(param, 
                xg_train, 
                num_round, 
                watchlist, 
                feval=lambda y,t: ("f1", f1_score(y, t.get_label(), average='micro')))

# get prediction
pred = bst.predict(xg_eval)
# error_rate = np.sum(pred != test_Y) / test_Y.shape[0]
F1 = f1_score(eval_Y, pred, average='micro')
print('Test error using softmax = {}'.format(F1))

In [ ]:
testsignals = []
with open("data/X_test.csv") as f_test:
    for line in f_test.readlines()[1:]:
        testsignals.append(list(map(int, line.split(',')[1:])))

In [ ]:
test_X = []
cnt = 0
for s in testsignals:
    print(cnt)
    cnt += 1
    beats = template(s)
    temp = [0 for i in range(len(beats[0]))]
    for beat in beats:
        for i, val in enumerate(beat):
            temp[i] += val
    for i, val in enumerate(temp):
        temp[i] = val / len(beats)
    test_X.append(temp)
test_X

In [ ]:
xg_test = xgb.DMatrix(test_X)
y_pred = bst.predict(xg_test)

In [ ]:
f = open("submission.csv", "w")
f.write("id,y\n")
for i,x in enumerate(y_pred):
    f.write("{},{}\n".format(i,y_pred[i]))
f.close()